In [2]:
%matplotlib inline
import importlib
import numpy as np
import pandas as pd
import json
import utils

In [3]:
importlib.reload(utils)

<module 'utils' from '/home/fei/Documents/projects/lyrics/encoder-decoder/utils.py'>

In [12]:
dat = pd.read_csv('../songdata.csv', usecols=['text']).drop_duplicates()

In [13]:
dat.head()

,text
0,"Look at her face, it's a wonderful face \nAnd..."
1,"Take it easy with me, please \nTouch me gentl..."
2,I'll never know why I had to go \nWhy I had t...
3,Making somebody happy is a question of give an...
4,Making somebody happy is a question of give an...


In [14]:
dat.loc[:, 'text'] += '(end)'

In [15]:
unique_chars = dat.loc[:, 'text'].apply(lambda x: list(set(x)))

In [16]:
unique_chars.head()

0    [a, z, A, s, l, p, S, f, b, n, o, ), y, i, (, ...
1    [a, z, A, s, p, l, f, b, n, o, ), O, y, i, (, ...
2    [a, A, l, p, s, S, B, f, b, n, o, ), y, i, (, ...
3    [a, A, s, p, l, S, B, f, b, n, o, O, ), y, i, ...
4    [a, A, s, p, l, S, B, f, b, n, o, O, ), y, i, ...
Name: text, dtype: object

In [17]:
unique_chars = unique_chars.sum()

In [18]:
unique_chars = sorted(np.unique(unique_chars))

In [19]:
len(unique_chars)

76

In [20]:
def scanning(text, length):
    encoder_input = []
    decoder_input = []
    text_length = len(text)
    for i in range(0, text_length-2*length+1):
        encoder_input.append(list(text[i:i+length]))
        decoder_input.append(list(text[i+length:i+2*length]))
    return encoder_input, decoder_input

In [ ]:
%%time
dat_scanned = dat.text.apply(lambda text: pd.Series(scanning(text, length=100), index=['for_encoder', 'for_decoder']))

In [12]:
dat_scanned.head()

,for_encoder,for_decoder
0,"[[L, o, o, k, , a, t, , h, e, r, , f, a, c,...","[[t, , s, h, e, , s, m, i, l, e, s, , w, h,..."
1,"[[T, a, k, e, , i, t, , e, a, s, y, , w, i,...","[[t, , s, l, o, w, , , \n, A, n, d, a, n, t..."
2,"[[I, ', l, l, , n, e, v, e, r, , k, n, o, w,...","[[a, c, k, i, n, g, , a, l, l, , m, y, , s,..."
3,"[[M, a, k, i, n, g, , s, o, m, e, b, o, d, y,...","[[ , y, o, u, r, s, e, l, f, , a, , b, r, e,..."
4,"[[M, a, k, i, n, g, , s, o, m, e, b, o, d, y,...","[[ , y, o, u, r, s, e, l, f, , a, , b, r, e,..."


In [13]:
%%time
for_encoder = dat_scanned.for_encoder.sum()
for_decoder = dat_scanned.for_decoder.sum()

CPU times: user 1min 2s, sys: 4.01 s, total: 1min 6s
Wall time: 1min 6s


In [14]:
%%time
for_encoder_se = pd.Series(for_encoder)
for_decoder_se = pd.Series(for_decoder)

In [15]:
for_encoder_se.head()

0    [L, o, o, k,  , a, t,  , h, e, r,  , f, a, c, ...
1    [o, o, k,  , a, t,  , h, e, r,  , f, a, c, e, ...
2    [o, k,  , a, t,  , h, e, r,  , f, a, c, e, ,, ...
3    [k,  , a, t,  , h, e, r,  , f, a, c, e, ,,  , ...
4    [ , a, t,  , h, e, r,  , f, a, c, e, ,,  , i, ...
dtype: object

In [16]:
for_decoder_se.head()

0    [t,  , s, h, e,  , s, m, i, l, e, s,  , w, h, ...
1    [ , s, h, e,  , s, m, i, l, e, s,  , w, h, e, ...
2    [s, h, e,  , s, m, i, l, e, s,  , w, h, e, n, ...
3    [h, e,  , s, m, i, l, e, s,  , w, h, e, n,  , ...
4    [e,  , s, m, i, l, e, s,  , w, h, e, n,  , s, ...
dtype: object

In [17]:
utils.save_unique_tokens(unique_chars)

In [18]:
word2ind, ind2word = utils.get_index_word_map(unique_chars)

In [19]:
utils.save_index_word_map(word2ind, ind2word)

In [20]:
encoder_input_tokenised = utils.tokenise_cleaned_data(for_encoder_se, word2ind)

In [21]:
encoder_input_tokenised.head()

0    [33, 64, 64, 60, 1, 50, 69, 1, 57, 54, 67, 1, ...
1    [64, 64, 60, 1, 50, 69, 1, 57, 54, 67, 1, 55, ...
2    [64, 60, 1, 50, 69, 1, 57, 54, 67, 1, 55, 50, ...
3    [60, 1, 50, 69, 1, 57, 54, 67, 1, 55, 50, 52, ...
4    [1, 50, 69, 1, 57, 54, 67, 1, 55, 50, 52, 54, ...
dtype: object

In [22]:
decoder_input_tokenised = utils.tokenise_cleaned_data(for_decoder_se, word2ind)

In [23]:
decoder_input_tokenised.head()

0    [69, 1, 68, 57, 54, 1, 68, 62, 58, 61, 54, 68,...
1    [1, 68, 57, 54, 1, 68, 62, 58, 61, 54, 68, 1, ...
2    [68, 57, 54, 1, 68, 62, 58, 61, 54, 68, 1, 72,...
3    [57, 54, 1, 68, 62, 58, 61, 54, 68, 1, 72, 57,...
4    [54, 1, 68, 62, 58, 61, 54, 68, 1, 72, 57, 54,...
dtype: object

In [24]:
decoder_target_tokenised = decoder_input_tokenised.copy()

In [25]:
decoder_target_tokenised.head()

0    [69, 1, 68, 57, 54, 1, 68, 62, 58, 61, 54, 68,...
1    [1, 68, 57, 54, 1, 68, 62, 58, 61, 54, 68, 1, ...
2    [68, 57, 54, 1, 68, 62, 58, 61, 54, 68, 1, 72,...
3    [57, 54, 1, 68, 62, 58, 61, 54, 68, 1, 72, 57,...
4    [54, 1, 68, 62, 58, 61, 54, 68, 1, 72, 57, 54,...
dtype: object

In [26]:
len(decoder_target_tokenised.loc[0])

100

In [27]:
decoder_input_tokenised = decoder_input_tokenised.apply(lambda x: [word2ind[' ']]+x[:-1])

In [28]:
decoder_input_tokenised.head()

0    [1, 69, 1, 68, 57, 54, 1, 68, 62, 58, 61, 54, ...
1    [1, 1, 68, 57, 54, 1, 68, 62, 58, 61, 54, 68, ...
2    [1, 68, 57, 54, 1, 68, 62, 58, 61, 54, 68, 1, ...
3    [1, 57, 54, 1, 68, 62, 58, 61, 54, 68, 1, 72, ...
4    [1, 54, 1, 68, 62, 58, 61, 54, 68, 1, 72, 57, ...
dtype: object

In [29]:
len(decoder_input_tokenised.iloc[0])

100

In [30]:
%%time
encoder_input_tokenised = pd.DataFrame(encoder_input_tokenised.values.tolist())
decoder_input_tokenised = pd.DataFrame(decoder_input_tokenised.values.tolist())
decoder_target_tokenised = pd.DataFrame(decoder_target_tokenised.values.tolist())

In [31]:
encoder_input_tokenised.shape

(1898365, 100)

In [32]:
decoder_input_tokenised.shape

(1898365, 100)

In [33]:
decoder_target_tokenised.shape

(1898365, 100)

In [34]:
encoder_input_tokenised.to_csv('encoder_input.csv', index=False)
decoder_input_tokenised.to_csv('decoder_input.csv', index=False)
decoder_target_tokenised.to_csv('decoder_target.csv', index=False)